# Import
---


In [0]:
import time
import pandas as pd

import torch
import torch.nn            as nn
import torch.nn.functional as F
import torch.optim         as optim

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets   as datasets

# Model
---
AlexNetを基にした小さい画像用CNNモデル

In [0]:
class CNN(nn.Module):
    
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(  1,  32, kernel_size=5, padding=2), nn.ReLU(inplace=True), nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d( 32,  64, kernel_size=5, padding=2), nn.ReLU(inplace=True), nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d( 64, 128, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1), nn.ReLU(inplace=True),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(), nn.Linear(256*6*6, 4096), nn.ReLU(inplace=True),
            nn.Dropout(), nn.Linear(   4096, 4096), nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(-1, 256*6*6) # flatten
        x = self.classifier(x)
        return x

In [0]:
if torch.cuda.is_available(): device = 'cuda'
else                        : device = 'cpu' 

net = CNN(num_classes=10).to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

# Load Dataset
---

In [70]:
toTensor  = transforms.ToTensor()
normalize = transforms.Normalize((0.5, ), (0.5, ))
transform = transforms.Compose([toTensor, normalize])

train_set = datasets.MNIST(root='../data', download=True, train=True , transform=transform)
valid_set = datasets.MNIST(root='../data', download=True, train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=1000, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=1000, shuffle=True)

print('size  :', train_set[0][0].size())
print('train :', len(train_set))
print('valid :', len(valid_set))

size  : torch.Size([1, 28, 28])
train : 60000
valid : 10000


# Training & Validation
---

In [0]:
def train():
    net.train()

    # mini-batch learning
    epoch_loss = 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        # forward
        t = net(x)
        # cal loss
        loss = loss_func(t, y)
        epoch_loss += loss.item()
        # backward
        loss.backward()
        # update model
        optimizer.step()
        optimizer.zero_grad()

    return epoch_loss

In [0]:
def valid():
    net.eval()

    with torch.no_grad():
        # mini-batch validation (for save memory)
        epoch_accu = 0
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            # forward
            t = net(x)
            # cal accuracy
            _, p = torch.max(t.data, dim=1)
            accu = (p == y).sum()
            epoch_accu += accu.item()

    return epoch_accu / len(valid_set)

In [0]:
max_epoch = 30

results = []
for epoch in range(max_epoch):
    st = time.time()
    loss = train()
    md = time.time()
    accu = valid()
    ed = time.time()

    results.append([epoch+1, md-st, loss, ed-md, accu])
    print(f'{epoch+1:>2} | {md - st:5.2f}s | {loss:>6.3f} | {ed - md:5.2f}s | {accu:.2%}')

 1 | 11.18s | 86.632 |  1.64s | 89.89%
 2 | 11.73s |  9.899 |  1.64s | 97.30%
 3 | 11.80s |  4.814 |  1.65s | 98.42%
 4 | 11.73s |  3.153 |  1.65s | 98.83%
 5 | 11.47s |  2.494 |  1.64s | 98.89%
 6 | 11.36s |  1.937 |  1.66s | 99.18%
 7 | 11.32s |  1.515 |  1.62s | 99.16%
 8 | 11.31s |  1.230 |  1.60s | 99.15%
 9 | 11.44s |  1.185 |  1.62s | 99.21%
10 | 11.53s |  1.241 |  1.63s | 99.36%
11 | 11.48s |  0.894 |  1.68s | 99.29%
12 | 11.47s |  0.749 |  1.61s | 99.28%
13 | 11.44s |  0.651 |  1.65s | 99.37%
14 | 11.44s |  0.641 |  1.58s | 99.33%
15 | 11.50s |  0.430 |  1.61s | 99.32%
16 | 11.44s |  0.468 |  1.61s | 99.27%
17 | 11.47s |  0.543 |  1.61s | 99.10%
18 | 11.46s |  0.488 |  1.71s | 99.31%
19 | 11.44s |  0.475 |  1.61s | 99.33%
20 | 11.45s |  0.410 |  1.62s | 98.95%
21 | 11.45s |  0.539 |  1.60s | 99.21%
22 | 11.41s |  0.368 |  1.63s | 99.33%
23 | 11.45s |  0.402 |  1.61s | 99.27%
24 | 11.46s |  0.522 |  1.60s | 99.11%
25 | 11.40s |  0.335 |  1.58s | 99.25%
26 | 11.44s |  0.324 |  1

# Save Model & Result
---

In [0]:
torch.save(net.state_dict(), 'models/MNIST.pt')

In [0]:
columns = ['epoch', 'train time(s)', 'loss', 'valid time(s)', 'accu']
df = pd.DataFrame(data=results, columns=columns).set_index('epoch')
df.to_csv('./results/MNIST.csv')

# Save Targets & Predicts
---

In [0]:
net.load_state_dict(torch.load('models/MNIST.pt'))

ps, ys = [], []
for x, y in valid_loader:
    x, y = x.to(device), y.to(device)

    t = net(x)
    _, p = torch.max(t.data, dim=1)

    ps.append(p.cpu())
    ys.append(y.cpu())

ps = torch.cat(ps).data.detach().numpy()
ys = torch.cat(ys).data.detach().numpy()

labels = datasets.MNIST.classes
ps = [labels[p] for p in ps]
ys = [labels[y] for y in ys]

In [0]:
df1 = pd.DataFrame(data={'target' :ys})
df2 = pd.DataFrame(data={'predict':ps})

df1.to_csv('outputs/target_MNIST.csv')
df2.to_csv('outputs/predict_MNIST.csv')